## Imports

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import DataFrame,SparkSession

## Funções EDA

In [0]:
def safra_aggroup_pct(df, col, name):
    agg_data = df.groupBy('safra', col).count()
    total_counts = agg_data.groupBy('safra').agg(F.sum('count').alias('total_count'))
    normalized_data = agg_data.join(total_counts, 'safra').withColumn('percentage', F.col('count') / F.col('total_count') * 100)

    pivot_data = normalized_data.groupBy('safra').pivot(col).sum('percentage').fillna(0).orderBy('safra').toPandas()

    pivot_data.set_index('safra', inplace=True)
    ax = pivot_data.plot(kind='bar', stacked=True, figsize=(20, 12))

    for container in ax.containers:
        ax.bar_label(container, fmt='%.0f%%', label_type='center')

    plt.xlabel('Safra')
    plt.ylabel('%')
    plt.title(f'{name} por Safra')
    plt.legend(loc = 'lower center', bbox_to_anchor=(1.05, 0), ncol=1, title=name)
    plt.show()

In [0]:
def safra_aggroup(df, col, name):
    agg_data = df.groupBy('safra', col).count()
    total_counts = agg_data.groupBy('safra').agg(F.sum('count').alias('total_count'))
    normalized_data = agg_data.join(total_counts, 'safra').withColumn('percentage', F.col('count') / F.col('total_count') * 100)

    pivot_data = normalized_data.groupBy('safra').pivot(col).sum('percentage').fillna(0).orderBy('safra').toPandas()

    pivot_data.set_index('safra', inplace=True)
    pivot_data.plot(kind='bar', stacked=True, figsize=(20, 12))

    plt.xlabel('Safra')
    plt.ylabel('%')
    plt.title(f'{name} por Safra')
    plt.legend(loc = 'lower center', bbox_to_anchor=(1.05, 0), ncol=1, title=name)
    plt.show()

## Bases

In [0]:
members = spark.table('sand_riscos_pm_pf.t730282_members')
user_logs = spark.table('sand_riscos_pm_pf.t730282_users_logs')
transactions = spark.table('sand_riscos_pm_pf.t730282_transactions')

# Análise Inicial

In [0]:
transactions.printSchema()

In [0]:
members.printSchema()

In [0]:
user_logs.printSchema()

## Transaction
Safra: 201501-201702

In [0]:
transactions.groupBy('safra').count().display()

In [0]:
transactions.select('msno', 'safra').distinct().groupBy('safra').count().display()

Na Transactions, só há um pagamento por safra!

Faz sentido.

In [0]:
safra_aggroup(transactions, 'payment_method_id', 'Payment Method ID')

In [0]:
safra_aggroup(transactions, 'payment_plan_days', 'Payment Plan Days')

In [0]:
safra_aggroup(transactions, 'plan_list_price', 'Plan List Price')

In [0]:
safra_aggroup(transactions, 'actual_amount_paid', 'Actual Amount Paid')

In [0]:
safra_aggroup(transactions, 'is_auto_renew', 'Is Auto Renew')

In [0]:
safra_aggroup(transactions, 'is_cancel', 'Is Cancell')

## Members
Safra: 201601-201612

In [0]:
members.groupBy('safra').count().display()

In [0]:
members.count(), members.select('msno').distinct().count()

In [0]:
members.drop('safra').dropDuplicates().count()

In [0]:
members

In [0]:
safra_aggroup(members, 'city', 'City')

In [0]:
# safra_aggroup(members, 'bd', 'Age')

In [0]:
members.groupBy('bd').count().display()

In [0]:
safra_aggroup(members, 'gender', 'Gender')

In [0]:
safra_aggroup(members, 'is_ativo', 'Is Active')

## User Logs
Safra: 201501-201702

In [0]:
user_logs.groupBy('safra').count().display()

In [0]:
user_logs.select('msno', 'safra').distinct().groupBy('safra').count().display()

No User_Logs, só há um dado por usuário por safra!

Faz sentido.

In [0]:
num_sec = ['num_25',
 'num_50',
 'num_75',
 'num_985',
 'num_100',
 'num_unq',
 'total_secs']

In [0]:
for k in num_sec:
    user_logs.select(k).describe().display()

In [0]:
user_logs.select('total_secs').summary().display()

In [0]:
user_logs.groupBy('safra').agg(F.min('total_secs').alias('min_total_secs'), F.max('total_secs').alias('max_total_secs'), (F.max('total_secs')/32140800).alias('pct')).display()

In [0]:
9.223372036927922E15/(31*24*60*60)

In [0]:
12*31*24*60*60

In [0]:
user_logs.filter(F.col('total_secs')>2678400).groupBy('safra').count().display()

In [0]:
user_logs.filter(F.col('total_secs')<0).groupBy('safra').count().display()

In [0]:
user_logs.filter(F.col('total_secs')>=0).filter(F.col('total_secs')<=2678400).groupBy('safra').agg(F.min('total_secs').alias('min_total_secs'), F.max('total_secs').alias('max_total_secs')).display()

## Target

In [0]:
user_logs.printSchema()

In [0]:
user_logs = (user_logs
             .withColumn('total_secs', F.when(F.col('total_secs')<0, 0)
                         .when(F.col('total_secs')>2678400, 2678400)
                         .otherwise(F.col('total_secs'))
             )
)
user_logs.printSchema()

In [0]:
transactions = (transactions
                .withColumn("payment_method_id", F.col("payment_method_id").cast("int"))
                .withColumn("payment_plan_days", F.col("payment_plan_days").cast("int"))
                .withColumn("plan_list_price", F.col("plan_list_price").cast("int"))
                .withColumn("actual_amount_paid", F.col("actual_amount_paid").cast("int"))
                .withColumn("is_auto_renew", F.col("is_auto_renew").cast("int"))
                .withColumn("transaction_date", F.to_date(F.col("transaction_date"), "yyyyMMdd"))
                .withColumn("membership_expire_date", F.to_date(F.col("membership_expire_date"), "yyyyMMdd"))
                .withColumn("is_cancel", F.col("is_cancel").cast("int"))
                )
transactions.printSchema()

In [0]:
members = (members
           .withColumn("safra", F.col("safra").cast("int"))
           .withColumn("registration_init_time", F.to_date(F.col("registration_init_time"), "yyyyMMdd"))
           .withColumn("city", F.col("city").cast("int"))
           .withColumn("bd", F.col("bd").cast("int"))
           .withColumn("registered_via", F.col("registered_via").cast("int"))
           .withColumn("safra_init", F.concat(F.substring(F.col("registration_init_time"),0,4),F.substring(F.col("registration_init_time"),6,2)).cast("int"))
           )
members.printSchema()

In [0]:
# window = Window.partitionBy("msno").orderBy("transaction_date")

# msno_target = (transactions
#                 .withColumn("next_transaction", F.lead("transaction_date").over(window))
#                 .withColumn("churn_window_end", F.add_months(F.col("membership_expire_date"), 3))
#                 )

# target = (msno_target.withColumn("target", F.when(
#     (F.col("next_transaction").isNull()) |
#     (F.col("next_transaction") > F.col("churn_window_end")) |
#     (F.col("is_cancel") == 1), 1).otherwise(0))
#               )

In [0]:
window = Window.partitionBy("msno").orderBy("transaction_date")

msno_target = (transactions
                .withColumn("next_transaction", F.lead("transaction_date").over(window))
                .withColumn("churn_window_end", F.add_months(F.col("membership_expire_date"), 3))
                )

target = (msno_target.withColumn("target", F.when(
    (F.col("next_transaction").isNull()) |
    (F.col("next_transaction") > F.col("churn_window_end")), 1).otherwise(0))
              )

In [0]:
target.filter(F.col('target').isin(1)).select('msno', 'target', 'transaction_date', 'next_transaction', 'churn_window_end').display()

In [0]:
target.filter(F.col('msno').isin('++UrWpg7IekInPcWfxrDFdkYSS4saPUwYxrqcoirTbg=')).select('msno', 'target', 'transaction_date', 'next_transaction', 'churn_window_end').display()

In [0]:
target.filter(F.col('msno').isin('++ZahLG+SvfBCljmIaj/KNOo4/ueRqnvLi2/gARCdLs=')).select('msno', 'target', 'transaction_date', 'next_transaction', 'churn_window_end').display()

In [0]:
#VERSÃO OFICIAL
base_final_target = ((target.join(user_logs, on=['msno', 'safra'], how='left')
                    .join(members.drop('safra'), on=['msno'], how='left')
                    ).filter(F.col('safra').between(201501, 201611))
                    .filter(F.col('is_ativo').isin(1))
                    .drop(*['next_transaction', 'churn_window_end'])).dropDuplicates()
base_final_target

In [0]:
# #VERSÃO V2
# base_final_target = ((target.join(user_logs, on=['msno', 'safra'], how='left')
#                     .join(members.drop('safra'), on=['msno'], how='left')
#                     ).filter(F.col('safra').between(201501, 201611))
#                     .drop(*['next_transaction', 'churn_window_end'])).dropDuplicates()
# base_final_target

**Data Leakage**

A variável is_cancel deve ser retirada da base, pois ela "vaza" informações da TARGET. 

is_ativo não é citado na documentação. Será excluída também.

# Análise da Base Final

## Churn Geral

In [0]:
base_final_target.groupBy('target').count().display()

In [0]:
total_churn = base_final_target.filter(F.col('target') == 1).count()
total_count = base_final_target.count()
churn_rate = round((total_churn / total_count) * 100, 2)
churn_rate

In [0]:
churn_safra = base_final_target.groupBy('safra').agg(F.round(F.avg(F.col('target')*100), 2).alias('churn_rate')).orderBy('safra')
churn_safra_df = churn_safra.toPandas()
churn_safra_df

In [0]:
plt.figure(figsize=(10, 6))
plt.bar([str(s) for s in churn_safra_df['safra']], churn_safra_df['churn_rate'], color='red')
plt.xlabel('Safra')
plt.ylabel('Churn Rate (%)')
plt.title('Taxa de Churn por Safra')
plt.xticks(rotation=45)
plt.show()

In [0]:
plt.figure(figsize=(14, 8))
bars = plt.bar([str(s) for s in churn_safra_df['safra']], churn_safra_df['churn_rate'], color='red')
plt.xlabel('Safra')
plt.ylabel('Churn Rate (%)')
plt.title('Taxa de Churn por Safra')
plt.xticks(rotation=45)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.1f}%', ha='center', va='bottom')

plt.show()

## Churn Auto Renew

In [0]:
is_auto_renew_rate = base_final_target.groupBy('safra').agg(F.round(F.avg(F.col('is_auto_renew')*100), 2).alias('is_auto_renew_rate')).orderBy('safra')
is_auto_renew_rate = is_auto_renew_rate.toPandas()
is_auto_renew_rate

In [0]:
base_final_target.filter(F.col('is_auto_renew').isin(0)).groupBy('target').count().display()

In [0]:
total_churn = base_final_target.filter(F.col('target') == 1).filter(F.col('is_auto_renew').isin(0)).count()
total_count = base_final_target.count()
churn_rate = round((total_churn / total_count) * 100, 2)
churn_rate

In [0]:
base_final_target.filter(F.col('is_auto_renew').isin(1)).groupBy('target').count().display()

In [0]:
total_churn = base_final_target.filter(F.col('target') == 1).filter(F.col('is_auto_renew').isin(1)).count()
total_count = base_final_target.count()
churn_rate = round((total_churn / total_count) * 100, 2)
churn_rate

In [0]:
churn_safra = base_final_target.filter(F.col('is_auto_renew').isin(0)).groupBy('safra').agg(F.round(F.avg(F.col('target')*100), 2).alias('churn_rate')).orderBy('safra')
churn_safra_df = churn_safra.toPandas()
churn_safra_df

In [0]:
plt.figure(figsize=(14, 8))
bars = plt.bar([str(s) for s in churn_safra_df['safra']], churn_safra_df['churn_rate'], color='skyblue')
plt.xlabel('Safra')
plt.ylabel('Churn Rate (%)')
plt.title('Taxa de Churn por Safra')
plt.xticks(rotation=45)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.1f}%', ha='center', va='bottom')

plt.show()

In [0]:
churn_safra = base_final_target.filter(F.col('is_auto_renew').isin(1)).groupBy('safra').agg(F.round(F.avg(F.col('target')*100), 2).alias('churn_rate')).orderBy('safra')
churn_safra_df = churn_safra.toPandas()
churn_safra_df

In [0]:
plt.figure(figsize=(14, 8))
bars = plt.bar([str(s) for s in churn_safra_df['safra']], churn_safra_df['churn_rate'], color='olivedrab')
plt.xlabel('Safra')
plt.ylabel('Churn Rate (%)')
plt.title('Taxa de Churn por Safra')
plt.xticks(rotation=45)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.1f}%', ha='center', va='bottom')

plt.show()

## Outras Análises

In [0]:
base_final_target.groupby('safra').count().display()

In [0]:
init_vars = ['payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'is_auto_renew',
 'city',
 'bd',
 'gender',
 'registered_via']

In [0]:
for var in init_vars:
    base_final_target.na.fill('Desconhecido').groupby(var).count().display()

In [0]:
null_vars = ['city',
'bd',
'gender',
'registered_via']

base_final_target_null = base_final_target

for var in null_vars:
    base_final_target_null = base_final_target_null.withColumn(var, F.when(F.col(var).isNull(), 'Desconhecido').otherwise(F.col(var)))

In [0]:
for var in null_vars:
    base_final_target_null.groupby(var).count().display()

## Análises Base Final

In [0]:
safra_aggroup(base_final_target_null, 'is_auto_renew', 'Is Auto Renew')

## Is_Ativo e Is_Cancel
Ambos serão excluídos da base.

In [0]:
data = base_final_target.groupby('is_ativo').count().toPandas()
plt.bar(data['is_ativo'].astype(str), data['count'])
plt.xlabel('is_ativo')
plt.ylabel('Count')
plt.title('Distribuição')
plt.show()

In [0]:
data = base_final_target.groupby('is_cancel').count().toPandas()
plt.bar(data['is_cancel'].astype(str), data['count'])
plt.xlabel('is_cancel')
plt.ylabel('Count')
plt.title('Distribuição')
plt.show()

Preciso avaliar quem vai fazer parte do público de Modelagem

Avaliar:
1. payment_plan_days == 0
2. plan_list_price == 0 (Quem está usando o período grátis e quem pediu para cancelar e ganhou 3 meses grátis)
3. actual_amount_paid == 0

## payment_plan_days

In [0]:
base_final_target.filter(F.col('payment_plan_days').isin(0)).groupBy('safra', 'target').count().display()

In [0]:
churn_safra = base_final_target.filter(F.col('payment_plan_days').isin(0)).groupBy('safra').agg(F.round(F.avg(F.col('target')*100), 2).alias('churn_rate')).orderBy('safra')
churn_safra_df = churn_safra.toPandas()
churn_safra_df

In [0]:
plt.figure(figsize=(14, 8))
bars = plt.bar([str(s) for s in churn_safra_df['safra']], churn_safra_df['churn_rate'], color='olivedrab')
plt.xlabel('Safra')
plt.ylabel('Churn Rate (%)')
plt.title('Taxa de Churn por Safra')
plt.xticks(rotation=45)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.1f}%', ha='center', va='bottom')

plt.show()

## plan_list_price

In [0]:
base_final_target.filter(F.col('plan_list_price').isin(0)).groupBy('safra', 'target').count().display()

In [0]:
churn_safra = base_final_target.filter(F.col('plan_list_price').isin(0)).groupBy('safra').agg(F.round(F.avg(F.col('target')*100), 2).alias('churn_rate')).orderBy('safra')
churn_safra_df = churn_safra.toPandas()
churn_safra_df

In [0]:
plt.figure(figsize=(14, 8))
bars = plt.bar([str(s) for s in churn_safra_df['safra']], churn_safra_df['churn_rate'], color='olivedrab')
plt.xlabel('Safra')
plt.ylabel('Churn Rate (%)')
plt.title('Taxa de Churn por Safra')
plt.xticks(rotation=45)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.1f}%', ha='center', va='bottom')

plt.show()

## actual_amount_paid

In [0]:
base_final_target.filter(F.col('actual_amount_paid').isin(0)).groupBy('safra', 'target').count().display()

In [0]:
churn_safra = base_final_target.filter(F.col('actual_amount_paid').isin(0)).groupBy('safra').agg(F.round(F.avg(F.col('target')*100), 2).alias('churn_rate')).orderBy('safra')
churn_safra_df = churn_safra.toPandas()
churn_safra_df

In [0]:
plt.figure(figsize=(14, 8))
bars = plt.bar([str(s) for s in churn_safra_df['safra']], churn_safra_df['churn_rate'], color='olivedrab')
plt.xlabel('Safra')
plt.ylabel('Churn Rate (%)')
plt.title('Taxa de Churn por Safra')
plt.xticks(rotation=45)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.1f}%', ha='center', va='bottom')

plt.show()

## Churn Ativo/Não Ativo

In [0]:
base_final_target = ((target.join(user_logs, on=['msno', 'safra'], how='left')
                    .join(members.drop('safra'), on=['msno'], how='left')
                    ).filter(F.col('safra').between(201501, 201611))
                    .drop(*['next_transaction', 'churn_window_end'])).dropDuplicates()
base_final_target

In [0]:
safra_aggroup(base_final_target.filter(F.col('is_ativo').isNotNull()), 'is_ativo', 'Is Ativo')

In [0]:
churn_safra = base_final_target.groupBy('safra').agg(F.round(F.avg(F.col('target')*100), 2).alias('churn_rate')).orderBy('safra')
churn_safra_df = churn_safra.toPandas()
churn_safra_df

In [0]:
plt.figure(figsize=(14, 8))
bars = plt.bar([str(s) for s in churn_safra_df['safra']], churn_safra_df['churn_rate'], color='olivedrab')
plt.xlabel('Safra')
plt.ylabel('Churn Rate (%)')
plt.title('Taxa de Churn por Safra')
plt.xticks(rotation=45)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.1f}%', ha='center', va='bottom')

plt.show()

In [0]:
churn_safra = base_final_target.filter(F.col('is_ativo').isin(0)).groupBy('safra').agg(F.round(F.avg(F.col('target')*100), 2).alias('churn_rate')).orderBy('safra')
churn_safra_df = churn_safra.toPandas()
churn_safra_df

In [0]:
plt.figure(figsize=(14, 8))
bars = plt.bar([str(s) for s in churn_safra_df['safra']], churn_safra_df['churn_rate'], color='olivedrab')
plt.xlabel('Safra')
plt.ylabel('Churn Rate (%)')
plt.title('Taxa de Churn por Safra')
plt.xticks(rotation=45)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.1f}%', ha='center', va='bottom')

plt.show()

In [0]:
churn_safra = base_final_target.filter(F.col('is_ativo').isin(1)).groupBy('safra').agg(F.round(F.avg(F.col('target')*100), 2).alias('churn_rate')).orderBy('safra')
churn_safra_df = churn_safra.toPandas()
churn_safra_df

In [0]:
plt.figure(figsize=(14, 8))
bars = plt.bar([str(s) for s in churn_safra_df['safra']], churn_safra_df['churn_rate'], color='olivedrab')
plt.xlabel('Safra')
plt.ylabel('Churn Rate (%)')
plt.title('Taxa de Churn por Safra')
plt.xticks(rotation=45)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.1f}%', ha='center', va='bottom')

plt.show()

# Salvar Base

In [0]:
drop_vars = ['is_ativo', 'is_cancel']

In [0]:
# VERSÃO OFICIAL
# base_final_target_null.drop(*drop_vars).write.mode("overwrite").saveAsTable('sand_riscos_pm_pf.T789778_spine_target_dm')
print('sand_riscos_pm_pf.T789778_spine_target_dm')

In [0]:
# #VERSÃO V2
# base_final_target_null.write.mode("overwrite").saveAsTable('sand_riscos_pm_pf.T789778_spine_target_dm_v2')
print('sand_riscos_pm_pf.T789778_spine_target_dm_v2')

In [0]:
#VERSÃO V2
base_final_target_null.write.mode("overwrite").saveAsTable('sand_riscos_pm_pf.T789778_spine_target_dm_v3')
print('sand_riscos_pm_pf.T789778_spine_target_dm_v3')